In [ ]:
## Some Observations after Performing Analysis

# 1) As the first scatter plot shows, there is a very strong correlation between
#    distance from the equator (measured degrees latitude) and maximum temperatures.

# 2) The data plotted in the last grid shows that windspeeds tend to be higher
#    far north of the equator than they are far south of the equator.

# 3)  According to the data visually depicted in the 2nd and 3rd scatter plots,
#     there does not appear to be much of any relationship between distance from the
#     equator (in degrees latitude) and percentage of cloudiness or humidity.

# 4)  Generally, most locations in the sample have a greater than 60% level of humidity
#     regardless of the distance from the equator. It would be interesting to see if this
#     pattern was similar say 20 or 30 years ago, or even further back before global
#     warming became a hot topic (no pun intended).
##

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
import json
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../Output/weather_data.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
counter = 0

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        counter += 1

# Print the city count to confirm sufficient count
print(len(cities))



In [ ]:
# set a dataframe to hold variables
weather_data = pd.DataFrame({"Cities": cities})
weather_data.head()

## Perform API Calls

In [ ]:

# Starting URL for Weather Map API Call
City = []
Country = []
Datetime = []
Latitude = []
Temperature = []
Humidity = []
Cloudiness = []
Windspeed = []

print("Beginning Data Retrieval")
print("----------------------------------")
for index, row in weather_data.iterrows():
    
    url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

    city = row['Cities']
    
    # update address key value
    
    city_url = (f"{url}&q={city}")
    
    # make request
    
    weather_response = requests.get(city_url)
 
    
#     # print the weather_json url, avoid doing for public github repos in order to avoid exposing key
    print(f"Processing record {index}: {city}. The url is included below.")
    print(city_url)
    
    # convert to json
    
    weather_json = weather_response.json()
    print("------------------------------------------")
    
    try:
        city = weather_json['name']
        City.append(city)
        country = weather_json['sys']['country']
        Country.append(country)
        datetime = weather_json['dt']
        Datetime.append(datetime)
        lat = weather_json['coord']['lat']
        Latitude.append(lat)
        temp = weather_json['main']['temp_max']
        Temperature.append(temp)
        humid = weather_json['main']['humidity']
        Humidity.append(humid)
        cloud = weather_json['clouds']['all']
        Cloudiness.append(cloud)
        windy = weather_json['wind']['speed']
        Windspeed.append(windy)
        
        # save data to a csv file

    except (KeyError, IndexError):
    
        print("city not tracked in weather app")
    
        print("skipped")
              
        
print("----------------------------")
print("  Data Retrieval Completed  ")
print("----------------------------")

## Store the collected data into a DataFrame

In [ ]:
# Define a new DataFrame using the weather data collected.
weather_data_df = pd.DataFrame({"City": City, "Country": Country, 
                                "Datetime": Datetime, 
                                "Latitude": Latitude, "Max Temperature (F)": Temperature, "Humidity %": Humidity, 
                                "Cloudiness %": Cloudiness, "Windspeed (mph)": Windspeed})

weather_data_df.count()

In [ ]:
# copy the results into the previously defined csv file
weather_data_df.to_csv(output_data_file, index=False, header=True)
weather_data_df.head()

In [ ]:
# convert the timestamp in the data to determine the date and time (GMT) 
# the analysis was started
timestamp = weather_data_df.iloc[0,2]
print(timestamp)
converted_timestamp = pd.Timestamp(timestamp, unit='s')
print(converted_timestamp)

## Generate Scatter Plots to Visualize Results

In [ ]:

weather_data_df.plot(kind="scatter", x="Latitude", y="Max Temperature (F)", grid=True, 
                    figsize=(17, 13), title=f"City Latitude vs Maximum Temperature (F) at {converted_timestamp}")
plt.savefig("../Images/latitude_v_max_temp.png")
plt.show()

In [ ]:
weather_data_df.plot(kind="scatter", x="Latitude", y="Humidity %", grid=True, 
                    figsize=(17,13), title=f"City Latitude vs Humidity (%) at {converted_timestamp}")
plt.savefig("../Images/latitude_v_humidity.png")
plt.show()

In [ ]:
weather_data_df.plot(kind="scatter", x="Latitude", y="Cloudiness %", grid=True, 
                    figsize=(17,13), title=f"City Latitude vs Cloudiness (%) at  {converted_timestamp}")
plt.savefig("../Images/latitude_v_cloudiness.png")
plt.show()

In [ ]:
weather_data_df.plot(kind="scatter", x="Latitude", y="Windspeed (mph)", grid=True, 
                    figsize=(17,13), title=f"City Latitude vs Windspeed (mph) at  {converted_timestamp}")
plt.savefig("../Images/latitude_v_windspeed.png")
plt.show()